<div style="background-color:#000;"><img src="pqn.png"></img></div>

### Set up the ArcticDB connection and helper functions

First, we establish a connection to the ArcticDB database and define a helper function to construct API URLs.

In [ ]:
import io
import os
import time
import requests
import pandas as pd
import arcticdb as adb

In [ ]:
arctic = adb.Arctic("lmdb://fundamantals")
lib = arctic.get_library("financial_ratios", create_if_missing=True)

In [ ]:
def build_fmp_url(request, period, year):
    apikey = os.environ.get("FMP_API_KEY")
    return f"https://financialmodelingprep.com/api/v4/{request}?year={year}&period={period}&apikey={apikey}"

We initiate a connection to the ArcticDB, a database designed for handling time-series data efficiently. We specify the database location using a URL and connect to a library within the database called "financial_ratios". The helper function `build_fmp_url` constructs URLs for API requests to the Financial Modeling Prep (FMP) service. It uses environmental variables to retrieve the API key securely, ensuring sensitive data is not hard-coded into the script.

### Retrieve financial data from the API

Next, we define a function that retrieves financial data from the FMP API and converts it into a pandas DataFrame.

In [ ]:
def get_fmp_data(request, period, year):
    url = build_fmp_url(request, period, year)
    response = requests.get(url)
    csv = response.content.decode("utf-8")
    return pd.read_csv(io.StringIO(csv), parse_dates=True)

In [ ]:
ratios = get_fmp_data("ratios-bulk", "quarter", "2020")
ratios

This function fetches data from the FMP API using the URL constructed by `build_fmp_url`. It sends a GET request to the API and receives a CSV formatted response containing financial ratios for companies. The CSV data is decoded and then read into a pandas DataFrame for easier manipulation and analysis. The function is called with specific parameters to retrieve data for the year 2020, and the resulting DataFrame is stored in the `ratios` variable.

### Store financial data in ArcticDB

Now, we iterate over multiple years to store their financial ratios into the ArcticDB.

In [ ]:
for year in [2020, 2021, 2022]:
    ratios = get_fmp_data("ratios-bulk", "quarter", year)
    adb_sym = f"financial_ratios/{year}"
    adb_fcn = lib.update if lib.has_symbol(adb_sym) else lib.write
    adb_fcn(adb_sym, ratios)
    time.sleep(10)

We loop over a list of years from 2020 to 2022, retrieving financial data for each year using the `get_fmp_data` function. For each year, we construct a unique symbol name to store the data in ArcticDB. We check if the symbol already exists in the library; if it does, we update it with the new data, otherwise, we write a new entry. A short delay is introduced between API requests to avoid overwhelming the server with requests in quick succession.

### Filter and query financial data

Finally, we define a function to filter financial data based on specific criteria and return the results as a pandas DataFrame.

In [ ]:
def filter_by_year(year):
    cols = [
        "symbol",
        "period",
        "date",
        "debtEquityRatio", 
        "currentRatio", 
        "priceToBookRatio", 
        "returnOnEquity", 
        "returnOnAssets", 
        "interestCoverage"
    ]
    q = adb.QueryBuilder()
    filter = (
        (q["debtEquityRatio"] < 0.5)
        & (
            (q["currentRatio"] > 1.5) & (q["currentRatio"] < 2.5)
        )
        & (q["priceToBookRatio"] < 1.5)
        & (q["returnOnEquity"] > 0.08)
        & (q["returnOnAssets"] > 0.06)
        & (q["interestCoverage"] > 5)
    )
    q = q[filter]
    return lib.read(f"financial_ratios/{year}", query_builder=q).data[cols].set_index("symbol")

In [ ]:
filter_by_year("2020")

This function filters financial data for a specified year based on certain financial metrics. It selects a subset of columns to focus on key financial ratios. The `QueryBuilder` is used to construct a query that filters data based on predefined criteria, such as debt-to-equity ratio and return on equity. The data is read from ArcticDB using this query, and the results are returned as a pandas DataFrame with the company symbol as the index, allowing easy access and analysis.

### Your next steps

Try changing the filtering criteria to explore different financial metrics. For example, adjust the debt-to-equity ratio threshold to see how it impacts the results. Experiment with different years to compare financial health across time. Exploring these variations will help you gain a deeper understanding of the financial data.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.